## Analise logs SATD

Funcionalidades: 
    
    
    Analise do gprof 
    
        Por aproximação
               Por vídeo e configuração
                -- Funções mais utilizadas (mais chamadas) (por QP)
                   - com gráfico
                -- Funções que gastam mais tempo (por QP)


    Analise do VVC log
        
        Por aproximação 
            -- bitrate
            -- PSNR
            -- tempo gasto 
    

In [12]:
from matplotlib import pyplot as plt
from IPython.display import clear_output,display, HTML
import numpy as np
import pandas as pd
import ipywidgets as widgets
import os
import glob

In [13]:
#CONSTANTS
GPROF_DIR = "../outputTests/gprof_log/"
VVCLOG_DIR =  "../outputTests/vvc_log/"


In [14]:
#CLASSES
#class LogFunction: 
class LogVideo:
    def __init__(self, config, qp, name, path):
        self.config = config
        self.QP = qp
        self.name = name
        self.path = path
        #self.m_used_functions = []
        self.time = None
        self.bitrate = None
        self.BD_rate = None
        self.PSNR = None
            
    def get_config(self):
        return self.config
            
    def get_QP(self):
        return self.QP
        
    def get_name(self):   
        return self.name
    
    def set_time(self,time):
        self.time = time
       
    def get_time(self):
        return self.time
        
    def set_bdrate(self, bdrate):
        self.BD_rate = bdrate
            
    def get_bdrate(self):
        return self.BD_rate
    
    def set_bitrate(self, bitrate):
        self.bitrate = bitrate
            
    def get_bitrate(self):
        return self.bitrate
            
    def set_PSNR(self, PSNR):
        self.PSNR = PSNR
            
    def get_PSNR(self):
        return self.PSNR
   
        
class Video:
    def __init__(self, name, path):
        self.name = name 
        self.path = path
        self.logs_AI = []
        self.logs_RA = []
        self.logs_LB = []
        
    def get_name(self): 
        return self.name

    def get_path(self):
        return self.path
    
    def get_logs(self, config):
        match config:
            case "AI":
                return self.logs_AI
            case "RA":
                return self.logs_RA
            case "LB":
                return self.logs_LB
            case _:
                return false           
            
    ###
    #funcao add logs config
    # vai na pasta, cria os logVideo e preenche
            
    def add_logs(self, config):
        path_logs = self.path + config + '/'
        
        #list logs in a specif configuration folder
        logs_list = os.listdir(path_logs)
        
        #fill config logs list
        #percorre os logs e vai preenchendo de acordo com o q ta na pasta 
        for log_item in logs_list:
            qp_item = log_item.split("_")[2]  
            qp_item_n = qp_item.split("p")[1]
            
            log_video = LogVideo(config,qp_item_n, log_item, path_logs)
        
            #aq eu preciso chamar outras funcoes pra preencher o resto dos dados dos logs
            #aquelas funcoes q pegam funcoes mais chamadas, tempo etc ect 
            get_data_vvc(path_logs + log_item, log_video)
            #print(log_video.get_bitrate())
            #print(log_video.get_PSNR())
            #print(log_video.get_time())    
        
            #coloca na lista
            match config:
                case "AI":
                    self.logs_AI.append(log_video)
                case "RA":
                    self.logs_RA.append(log_video)
                case "LB":
                    self.logs_LB.append(log_video)    
    
class Aproximation:
    def __init__(self, size, aprox_type, path):
        self.size = size
        self.aprox_type = aprox_type
        self.path = path
        self.videos = []

    def add_video(self, video):
        self.videos.append(video)
        
    def get_size(self):
        return self.size
    
    def get_type(self):
        return self.aprox_type
    
    def get_path(self):
        return self.path
    
    #returns list
    def get_videos(self):
        return self.videos

class Matrix: 
    def __init__(self, size):
        self.size = size
        self.aprox_list = []
        
    #add a new aprox (-1, -2, -3...)
    def add_aprox(self, aprox):
        self.aprox_list.append(aprox)
        
    def get_aproximations(self):
        return self.aprox_list
    

In [15]:
#FUNCTIONS SYSTEM
#list all matrices
def list_matrices(terminator, folder):
    items = os.listdir(folder)
    names = []
    for item in items:
        if os.path.isdir(os.path.join(folder, item)):
            prefix = item.split(terminator)[0]  
            if prefix not in names:  
                names.append(prefix)
    return names
    
#list all aproximations of a matrix researched
def list_aproximations(prefix, terminator, folder):
    items = os.listdir(folder)
    names = []
    for item in items:
        if os.path.isdir(os.path.join(folder, item)):
            prefix_item = item.split(terminator)[0]
            if prefix_item == prefix: 
                names.append(item)
    return names
    
    
#list all videos from a aprox folder
def list_videos(aprox_folder):
    videos = os.listdir(aprox_folder)
    return videos
    

In [16]:
#FUNCTIONS GPROF LOG
#returns the most used functions list 
def get_m_used_functions(file_gprof, number_of_itens):
    #--> ordenar funcoes de acordo com o nome (tipo aquele filtro) e devolver o objeto funcoes
    return void

 

In [17]:
#FUNCTIONS VVCLOG
def get_data_vvc(file_path, video_log):
    with open(file_path, 'r') as f:
        next = False
        for line in f:
            if "Total Frames" in line:
                next = True
            
            else: 
                if next:
                    data = ((line.strip().split("a")[1]).strip()).split()
                    
                    #teste = data.split()
                    video_log.set_bitrate(data[0])
                    
                    #PSNR YUV(??)
                    video_log.set_PSNR(data[4])

                    next = False
                    
            if "Total Time" in line:
                #elapsed(??)
                time = (line.strip().split()[2])
                video_log.set_time(time)

In [39]:
#FUNCTIONS PRINT TABLE

#make the table according the param
def print_table_vvc(matrix, videos):
    tags_list = []
    aprox_list = []
    videos_list = []
    aproximations = matrix.get_aproximations()
    for aproximation in aproximations:
        aprox_list.append(aproximation.get_type())
    
    for video in videos:
        print(video)
        videos_list.append(video)
        videos_list.append(video)
        videos_list.append(video)

        tags_list.append("time")
        tags_list.append("bitrate")
        tags_list.append("PSNR")
        
    
    #imprime tabela dos videos
    df = pd.DataFrame(videos_list, tags_list, columns=[("video","parametros")])
    display(df)

def print_table_gprog():
    return void

In [40]:
#FUNCTIONS
#returns the analise
def analise(matrix_size, folder):
    
    #new matrix according dropdown
    matrix = Matrix(matrix_size)
    
    #find all aproximations for the matrix and fill object
    aproximations_list = list_aproximations(matrix_size, "-",folder)

    for aproximation in aproximations_list:
        aprox_type = aproximation.split("-")[2]
        aprox_folder = folder + aproximation + "/"
        
        #new object Aproximation
        aprox = Aproximation(matrix_size, aprox_type, aprox_folder)
        
        #fill aproximation videos list
        videos_list = list_videos(aprox.get_path())
        for item in videos_list:
            video = Video(item, aprox.get_path() + item + '/')  
            
            #fill logs for the video
            configs_list = os.listdir(aprox.get_path() + item + '/')
            for item_config in configs_list:
                video.add_logs(item_config)
            
            aprox.add_video(video)
                    
        #add video to matrix
        matrix.add_aprox(aprox)
    
    print_table_vvc(matrix, videos_list)
    #TESTE
    #usar esse teste como base pra montar a tabela depois
    #analise chama funcao q faz dataframe com os dados e que retorna dataframe
    #analise retorna esse dataframe 
    #botao chama funcao que faz tabela com esse dataframe 
    
    #aprox_test = matrix.get_aproximations()
    #itera as aproximacoes 
    #for aprox_item in aprox_test:
    #    print("-" + aprox_item.get_type())
            
        #itera os videos dentro das aproximacoes
        #video_test = aprox_item.get_videos()
        #for video_item in video_test:
            #print(video_item.get_name() + ":")

            #logs_test = video_item.get_logs("AI")
            #for log_item in logs_test:
                #print(log_item.get_QP())

In [41]:
#FUNCTIONS BUTTON

#geral
def on_button_clear_click(b):
    clear_output()
    analise_vvc_print()
#vvc
def on_button_print_click(b):
    #listando todos os videos dentro das pastas das aproximacoes do valor aproximado
    analise(dropdown.value, VVCLOG_DIR)  
    

 

In [42]:
# widgets VVC 
def analise_vvc_print():
    display(HTML("<h2>Analise VVC</h2>"))
    display(dropdown, botao, botao_clear)

#widgets GPROF
def analise_gprof_print():
    display(HTML("<h2> Analise GPROF</h2>"))
    

In [43]:
#MAIN 
# Dropdown
# preciso adicionar opcao de comparar com o PRECISE depois
# preciso ver quando vou mandar VVC e quando vou mandar GPROF
options_drop = list_matrices("-", VVCLOG_DIR)
options_drop.remove("Precise")
dropdown = widgets.Dropdown(options=options_drop)

# Buttons
botao = widgets.Button(description='Imprimir')
botao.on_click(on_button_print_click)

botao_clear = widgets.Button(description='Clear')
botao_clear.on_click(on_button_clear_click)

# Print
analise_vvc_print()


Dropdown(options=('16x8', '8x16'), value='16x8')

Button(description='Imprimir', style=ButtonStyle())

Button(description='Clear', style=ButtonStyle())

BasketballDrive
BasketballPass
BQMall
Campfire
FoodMarket4
FourPeople
Johnny
RaceHorses
RaceHorses2
RitualDance


,"(video, parametros)"
time,BasketballDrive
bitrate,BasketballDrive
PSNR,BasketballDrive
time,BasketballPass
bitrate,BasketballPass
PSNR,BasketballPass
time,BQMall
bitrate,BQMall
PSNR,BQMall
time,Campfire
